In [1]:
#Import libraries
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os
import random
import joblib
import tensorflow as tf
from tensorflow.keras.layers import Input,LSTM,Dropout,Dense,Bidirectional,Conv2D,Embedding,Masking,TimeDistributed,MaxPool2D
from tensorflow.keras.layers import Reshape,Lambda,Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,LearningRateScheduler,ReduceLROnPlateau
import tensorflow.keras.backend as K
from sklearn.metrics import confusion_matrix

<h1> 1. Load utilities

In [2]:
[_,_,_,_,_,_,embedding_matrix,tokenizer,label_index] = joblib.load('utils')

In [3]:
index_to_label = {j:i for i,j in label_index.items()}
print('Index Label:',index_to_label)
classes = len(label_index)
print('No of classes:',classes)
vocab_size = embedding_matrix.shape[0]
print('Vocab size:',vocab_size)
#Maximum words in any utterance
max_sentence_length = 35
#Maximum utterances in a dialogue
max_utterances = 24

Index Label: {1: 'neutral', 2: 'joy', 3: 'surprise', 4: 'anger', 5: 'sadness', 6: 'fear', 7: 'disgust'}
No of classes: 7
Vocab size: 5243


<h1>2. Load Model

In [4]:
def get_model():
    
    inputs = Input(shape=(max_utterances,max_sentence_length),name='Input_layer',dtype='int32')

    #Extract one utterance
    def slicer(x, index):
        return x[:,K.constant(index, dtype='int32'),:]
    def slicer_output_shape(input_shape):
        #shape == (Batch size, utterances, sentence_length)
        shape = list(input_shape)
        new_shape = (shape[0], shape[2])
        return new_shape

    #Add one dimension to embedding layer output
    def reshaper(x):
            return K.expand_dims(x, axis=3)

    #Flatten
    def flattener(x):
        x = K.reshape(x, [-1, x.shape[1]*x.shape[3]])
        return x
    def flattener_output_shape(input_shape):
        shape = list(input_shape)
        #Multipy by 3 due to 3 filters
        new_shape = (shape[0], 3*shape[3])
        return new_shape

    embedding = Embedding(input_dim=vocab_size,output_dim=300,input_length=max_sentence_length,
                          name='Embedding_layer',trainable=False,weights=[embedding_matrix])
    
    conv_0 = Conv2D(512, kernel_size=(3,300), padding='valid', kernel_initializer='normal', activation='relu',name='Conv2D_0')
    conv_1 = Conv2D(512, kernel_size=(4,300), padding='valid', kernel_initializer='normal', activation='relu',name='Conv2D_1')
    conv_2 = Conv2D(512, kernel_size=(5,300), padding='valid', kernel_initializer='normal', activation='relu',name='Conv2D_2')
    maxpool_0 = MaxPool2D(pool_size=(max_sentence_length - 3 + 1, 1), strides=(1,1), padding='valid',name='MaxPool2D_0')
    maxpool_1 = MaxPool2D(pool_size=(max_sentence_length - 4 + 1, 1), strides=(1,1), padding='valid',name='MaxPool2D_1')
    maxpool_2 = MaxPool2D(pool_size=(max_sentence_length - 5 + 1, 1), strides=(1,1), padding='valid',name='MaxPool2D_2')
    
    dense_func = Dense(512,activation='tanh',name='Dense_1')
    drop = Dropout(0.5,name='Dropout_1')

    cnn_output = []
    for i in range(max_utterances):
        # Extract utterance of dialogue
        utter = Lambda(slicer, output_shape=slicer_output_shape, arguments={"index":i},
                       name='Utterance_{}'.format(i))(inputs)
        # Embedding layer
        embed = embedding(utter)
        # Expand dimensions 
        reshape = Lambda(reshaper,name='Reshape_Utterance_{}'.format(i))(embed)
        # Concatenate outputs from three filters
        concatenate = Concatenate(axis=1,name='Concat_Utterance_{}'.format(i))([maxpool_0(conv_0(reshape)),
                                                                                maxpool_1(conv_1(reshape)),
                                                                                maxpool_2(conv_2(reshape))])
        # Flatten
        flatten = Lambda(flattener, output_shape=flattener_output_shape,name='Flatten_Utterance_{}'.format(i))(concatenate)
        # 512 dimensional vector for each utterances
        dense_output = dense_func(flatten)
        dropout = drop(dense_output)
        cnn_output.append(dropout)

    def stack(x):
        return K.stack(x, axis=1)
    # Stack cnn_output along axis 1 (timestep/utterance axis)
    cnn_outputs = Lambda(stack,name='CNN_outputs')(cnn_output)
    # Mask utterances which are zero padded
    masked = Masking(mask_value=0,name='Masking')(cnn_outputs)
    # 1 layer of Bidirectional LSTMs
    lstm = LSTM(256, return_sequences = True, dropout=0.5, name='LSTM_1')(masked)
    lstm = Dropout(0.5,name='Dropout_2')(lstm)
    # Output layer for each utterances
    # Class label from numbered from 1, hence 1 is added
    output = Dense(classes+1,name='Output_layer')(lstm)
    
    model = Model(inputs, output,name='bc-LSTM')
    return model

In [5]:
#Build model
model = get_model()
#Load weights
model.load_weights('best_model1.h5')

<h1>3. Predict Function

In [6]:
def get_emotion(dialogue):
    
    dialogue_length = len(dialogue)
    
    #Preprocess Utterances
    #Decontraction of text
    def decontracted(phrase):
        """
        Returns decontracted phrases
        """
        # specific
        phrase = re.sub(r"won't", "will not", phrase)
        phrase = re.sub(r"can\'t", "can not", phrase)
        # general
        phrase = re.sub(r"n\'t", " not", phrase)
        phrase = re.sub(r"\'re", " are", phrase)
        phrase = re.sub(r"\'s", " is", phrase)
        phrase = re.sub(r"\'d", " would", phrase)
        phrase = re.sub(r"\'ll", " will", phrase)
        phrase = re.sub(r"\'t", " not", phrase)
        phrase = re.sub(r"\'ve", " have", phrase)
        phrase = re.sub(r"\'m", " am", phrase)
        return phrase
    def preprocess(sentence):
    
        # process sentence
        sentence = re.sub('\x92','\'',sentence)
        # decontract sentence
        sentence = decontracted(sentence)
        # creating a space between a word and the punctuation following it
        sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
        sentence = re.sub(r'[" "]+', " ", sentence)
        # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
        sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
        # remove extra spaces
        sentence = sentence.strip()
        # make lower case
        sentence = sentence.lower()
        return sentence
    dia = np.array([preprocess(i) for i in dialogue])
    
    #Tokenize Utterances
    def tokenizer_sentence(sentence,pad):
        encoded_docs = tokenizer.texts_to_sequences([sentence])
        padded_docs = tf.keras.preprocessing.sequence.pad_sequences(encoded_docs,maxlen=pad,padding='post')
        return padded_docs
    def tokenize_dialogue(dialogue,max_sentence_length,max_utterances):
        utt = []
        for i in dialogue:
            utt.append(tokenizer_sentence(i,pad=max_sentence_length))
        p = np.array(utt).reshape(-1,max_sentence_length)
        # Pad utterances
        q = np.zeros((max_utterances-len(utt),max_sentence_length),dtype='int8')
        dia_array = np.concatenate([p,q])
        return dia_array
    dialogue_token = tokenize_dialogue(dia,max_sentence_length,max_utterances)
    
    #Get predictions from model
    def predict_label(x):
        a = model.predict(np.expand_dims(x,0))
        return [i.argmax() for i in a[0]]
    labels = predict_label(dialogue_token)[:dialogue_length]
    
    return dialogue,[index_to_label[i] for i in labels]

<h1>4. Load dataset

In [7]:
test = pd.read_csv('MELD/test_sent_emo.csv')
test.head()

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,Why do all youre coffee mugs have numbers on ...,Mark,surprise,positive,0,0,3,19,"00:14:38,127","00:14:40,378"
1,2,Oh. Thats so Monica can keep track. That way ...,Rachel,anger,negative,0,1,3,19,"00:14:40,629","00:14:47,385"
2,3,Y'know what?,Rachel,neutral,neutral,0,2,3,19,"00:14:56,353","00:14:57,520"
3,19,"Come on, Lydia, you can do it.",Joey,neutral,neutral,1,0,1,23,"0:10:44,769","0:10:46,146"
4,20,Push!,Joey,joy,positive,1,1,1,23,"0:10:46,146","0:10:46,833"


<h1>5. Predictions from Model

In [8]:
dialogue = test[test['Dialogue_ID']==25]['Utterance'].values
dialogue

array(['What? Honey.',
       'Oh, I am, my side still hurts from when you crashed into me yesterday.',
       'Oh God, I\x92m so sorry.', 'I know.', 'Ow!!', 'Oh God!',
       'Hey, you guys! Guess what?', 'Got a job on a river boat?',
       "Y'know what I didn\x92t wear this suit for a year because you hated it. Well, guess what? You\x92re not my girlfriend anymore so...",
       'Oh I see, so this suit is making a point.',
       'Now that you\x92re on you\x92re own, you\x92re free to look as stupid as you like.',
       'You like it right?',
       'Oh absolutely. I like it even more on you than I did on Colonel Sanders.  Ross! Ross! I\x92m kidding!',
       'Yeah, come here!', 'What-what was it you were gonna tell us?',
       'Yeah. Oh! Was how you invented the cotton gin?!',
       'Okay, good bye!'], dtype=object)

In [9]:
#Get predictions for dialogue
dialogue, labels = get_emotion(dialogue)
for i,j in zip(dialogue,labels):
    print('Utterance:',i)
    print('Emotion:',j)
    print(120*'-')

Utterance: What? Honey.
Emotion: neutral
------------------------------------------------------------------------------------------------------------------------
Utterance: Oh, I am, my side still hurts from when you crashed into me yesterday.
Emotion: neutral
------------------------------------------------------------------------------------------------------------------------
Utterance: Oh God, Im so sorry.
Emotion: sadness
------------------------------------------------------------------------------------------------------------------------
Utterance: I know.
Emotion: neutral
------------------------------------------------------------------------------------------------------------------------
Utterance: Ow!!
Emotion: surprise
------------------------------------------------------------------------------------------------------------------------
Utterance: Oh God!
Emotion: surprise
--------------------------------------------------------------------------------------------------

In [10]:
dialogue = test[test['Dialogue_ID']==85]['Utterance'].values
dialogue

array(['But um, I don\x92t think it\x92s anything serious.',
       'This sounds like a hernia. You have to\x97you-you\x97Go to the doctor!',
       'No way!',
       '\x91Kay look, if I have to go to the doctor for anything it\x92s gonna be for this thing sticking out of my stomach!',
       'Why did I have to start working out again?', 'Damn you 15s!'],
      dtype=object)

In [11]:
#Get predictions for dialogue
dialogue, labels = get_emotion(dialogue)
for i,j in zip(dialogue,labels):
    print('Utterance:',i)
    print('Emotion:',j)
    print(120*'-')

Utterance: But um, I dont think its anything serious.
Emotion: neutral
------------------------------------------------------------------------------------------------------------------------
Utterance: This sounds like a hernia. You have toyou-youGo to the doctor!
Emotion: anger
------------------------------------------------------------------------------------------------------------------------
Utterance: No way!
Emotion: anger
------------------------------------------------------------------------------------------------------------------------
Utterance: Kay look, if I have to go to the doctor for anything its gonna be for this thing sticking out of my stomach!
Emotion: anger
------------------------------------------------------------------------------------------------------------------------
Utterance: Why did I have to start working out again?
Emotion: neutral
--------------------------------------------------------------------------------------------------------------

In [12]:
dialogue = test[test['Dialogue_ID']==59]['Utterance'].values
dialogue

array(["You got the clothes clean. Now that's the important part.",
       'Oh, I guess. Except everything looks like jammies now.',
       "Whoa, I'm sorry. Excuse me. We had this cart."], dtype=object)

In [13]:
#Get predictions for dialogue
dialogue, labels = get_emotion(dialogue)
for i,j in zip(dialogue,labels):
    print('Utterance:',i)
    print('Emotion:',j)
    print(120*'-')

Utterance: You got the clothes clean. Now that's the important part.
Emotion: neutral
------------------------------------------------------------------------------------------------------------------------
Utterance: Oh, I guess. Except everything looks like jammies now.
Emotion: neutral
------------------------------------------------------------------------------------------------------------------------
Utterance: Whoa, I'm sorry. Excuse me. We had this cart.
Emotion: sadness
------------------------------------------------------------------------------------------------------------------------
